### *Import  libraries*




In [86]:
import os

import re
import nltk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import layers
from keras.backend import clear_session
from keras.models import load_model
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
import string
from google.colab import drive 
drive.mount('/content/gdrive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### *Import data and label encoding* 

In [87]:
data = pd.read_csv('/content/gdrive/MyDrive/corpusAAS.txt',sep='\n', names=['sent'])
data['y'] = data.sent.str[-3:]
data['sent'] = data.sent.str[:-4]

data['y'].replace('pos', 1, inplace=True)
data['y'].replace('neg', 0, inplace=True)

In [88]:
data.head(2000)

,sent,y
0,'طوال حياتي لم المس اي تغير حتى قدمت هذه الحكو...,1
1,'منتوج رائع وثمن مناسب ....جميل',1
2,'كلنا ابن كيران لمتافق معايا يدير جيم',1
3,'وفقك الله لولاية اخرى حقاش مكينش محسن منك',1
4,'لأنه و بكل بساطة رئيس الحكومة يعتني بمعاق داخ...,1
...,...,...
1995,'اصمت لعلى صمتك راحة بالنسبة لهم',0
1996,'حديقة حيوانات ولازال هنالك اناس لا يؤمنون بنظ...,0
1997,'أفعى بجدارة تريثت تربصت وكان الفحيح متعة له ص...,0
1998,'لا يقطع الرأس غير الي ركبه الان اصبح تركيب ال...,0


In [71]:
#ar_stopwords ="نفسنا مثل حيث ذلك بشكل لدى ألا عن إلي ب لنا وقالت فقط الذي الذى ا هذا غير أكثر اي أنا أنت ايضا اذا كيف وكل أو اكثر أي أن منه وكان وفي تلك إن سوف حين نفسها هكذا قبل حول منذ هنا عندما على ضمن لكن فيه عليه قليل صباحا لهم بان يكون بأن أما هناك مع فوق بسبب ما لا هذه و فيها ف ولم ل آخر ثانية انه من الان جدا به بن بعض حاليا بها هم أ كانت هي لها نحن تم أنفسهم ينبغي إلى فان وقد تحت عند وجود الى فأن الي او قد خارج إنه اى مرة هؤلاء أنها إذا فهي فهى كل يمكن جميع أنفسكم فعل كان ثم لي الآن وقال فى في ديك لم لن له تكون الذين ليس التى التي أنه وان بعد حتى ان دون وأن لماذا يجري كلا إنها لك ضد وإن فهو انها منها أى لديه ولا بين خلال وما اما عليها بعيدا كما نفسي نحو هو نفسك نفسه انت ولن إضافي لقاء وكانت هى فما أيضا إلا معظم ومن إما الا بينما وهي وهو وهى"

In [72]:
#ar_stopwords=nltk.word_tokenize(ar_stopwords)


A function to clean and preprocess data

In [89]:
def clean_text(text):  

    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى",
              "\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا",
               "","","","ي","",' ', ' ',' ',' ? ',' ؟ ', ' ! ']
    #remove diacritic
    diacritic = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(diacritic,"", text)
  
    longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    
    text = re.sub(r"[^\w\s]", '', text)
    #remove english words
    text = re.sub(r"[a-zA-Z]", '', text)
    #remove spaces
    text = re.sub(r"\d+", ' ', text)
    text = re.sub(r"\n+", ' ', text)
    text = re.sub(r"\t+", ' ', text)
    text = re.sub(r"\r+", ' ', text)
    text = re.sub(r"\s+", ' ', text)
    #remove repetetions
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    #remove longation
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    text = text.strip()
    stemmer = nltk.ISRIStemmer()
    word_list = nltk.word_tokenize(text)
    #remove arabic stopwords
    word_list = [ w for w in word_list if not w in stop_words ]
    #remove digits
    word_list = [ w for w in word_list if not w.isdigit() ]
    #stemming
    word_list = [stemmer.stem(w) for w in  word_list]
    text= ' '.join(word_list) 
    
    return text

In [91]:

punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation
stop_words = stopwords.words()

data['sent'] = data['sent'].apply(clean_text)
data.head(5)


,sent,y
0,طول حيت لمس اي تغر حتي قدم حكم فل نقف بجن بصت,1
1,نتج رءع وثم نسب جمل,1
2,كلن ابن كير تفق معا يدر,1
3,وفق الل لول اخر حقش كينش حسن منك,1
4,لنه بكل بسط رءس حكم يعت عاق دخل بيت الل ميز حسن,1


split data into train and test

In [78]:
feature = data.sent
target = data.y

In [79]:
X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size =.2, random_state=100)


In [94]:
print(X_train.shape)
print(X_test.shape)

(1600,)
(400,)


### *Modeling*

I will create 3 models for comparison 


1.   SVC
2.   LSTM
3.   CNN



### **I started by crating a TF IDF + SVC pipeline as a baseline**







In [81]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV


In [82]:
#create the  pipeline
pipe = make_pipeline(TfidfVectorizer(), SVC())
#Hyperparameter tuning using  GridSearch cross validation
param_grid = {'svc__kernel': ['rbf', 'linear', 'poly'],
             'svc__gamma': [0.1, 1, 10, 100],
             'svc__C': [0.1, 1, 10, 100]}

svc_model = GridSearchCV(pipe, param_grid, cv=3)
svc_model.fit(X_train, Y_train)

prediction = svc_model.predict(X_test)
#evaluation
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction))

Accuracy score is 0.84
              precision    recall  f1-score   support

           0       0.81      0.91      0.85       204
           1       0.89      0.78      0.83       196

    accuracy                           0.84       400
   macro avg       0.85      0.84      0.84       400
weighted avg       0.85      0.84      0.84       400



### **CNN Model**

I will use Keras Tokenizer to split each word in a sentence. Then, in order to get a sequential representation of each row we use texts_to_sequences method. after that, we check the first five entries and there sequential representation, where each word is represented by a number.

In [97]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)

for x in X_train[:5]:
    print (x)    
sequences[:5]

يحق ان يخد حرب علي تهل ربق
وفق ان شاء الل
الل كمل برك الل
اظن ان شعب غرب تبع رمج فظل غلق وسع نظر
عار عار


[[1729, 4, 863, 195, 2, 214, 1730],
 [10, 4, 21, 1],
 [1, 284, 19, 1],
 [683, 4, 11, 3, 258, 123, 1132, 684, 563, 27],
 [237, 237]]

Now, we can check the max length of rows in our corpus for padding.



In [27]:
length = []
for x in X_train:
    length.append(len(x.split()))
max(length)

70

We can say that the maximum length to be 80



In [30]:
x_train_seq = pad_sequences(sequences, maxlen=80)
x_train_seq[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0, 1773,    4,  895,  188,  208,
           2,  254, 1774],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           

After checking, we can see that all the data transformed to have the same length of 80.

We do the same thing to the test set.

In [32]:
sequences_test = tokenizer.texts_to_sequences(X_test)
x_test_seq = pad_sequences(sequences_test, maxlen=80)
x_test_seq[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,  223,   57,
         377,  887,  135,  360, 1486, 3636,  171,  576,  518,  449,   82,
         308,  387,  135,  192,  518, 1883, 2284, 2672, 1401,  718,  710,
         718,    2,   12],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           

Next, I defined a CNN using an embedding layer of 200x80 dimension as an input with 100000 as a max feature, then add to our 1D Convolutional layer 100x2000 filters, then add Global Max Pooling layer which will extract the maximum value from each filter. Finally, the output will be a one-dimensional vector with length equal to the number of the filters.

In [35]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding

model_cnn = Sequential()

e = Embedding(100000, 100, input_length=80)
model_cnn.add(e)
model_cnn.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(256, activation='relu'))
model_cnn.add(Dense(1, activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn.fit(x_train_seq, Y_train, validation_data=(x_test_seq, Y_test), epochs=5, batch_size=32, verbose=2)
score,acc = model_cnn.evaluate(x_test_seq, Y_test, verbose = 2, batch_size = 32)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

Epoch 1/5
50/50 - 5s - loss: 0.6666 - accuracy: 0.6294 - val_loss: 0.5866 - val_accuracy: 0.7800 - 5s/epoch - 97ms/step
Epoch 2/5
50/50 - 3s - loss: 0.3294 - accuracy: 0.8975 - val_loss: 0.3512 - val_accuracy: 0.8575 - 3s/epoch - 62ms/step
Epoch 3/5
50/50 - 3s - loss: 0.0898 - accuracy: 0.9719 - val_loss: 0.3942 - val_accuracy: 0.8450 - 3s/epoch - 61ms/step
Epoch 4/5
50/50 - 3s - loss: 0.0206 - accuracy: 0.9969 - val_loss: 0.4095 - val_accuracy: 0.8725 - 3s/epoch - 60ms/step
Epoch 5/5
50/50 - 3s - loss: 0.0069 - accuracy: 0.9994 - val_loss: 0.4705 - val_accuracy: 0.8500 - 3s/epoch - 61ms/step
13/13 - 0s - loss: 0.4705 - accuracy: 0.8500 - 58ms/epoch - 4ms/step
score: 0.47
acc: 0.85


After training, we get the above accuracy, which seems a better result from the model I've run in previous kernel SVC+TF-IDF.

### ***LSTM Model***


In this model will use a single LSTM layer preceded by an embedding layer with 100000 as a max feature and 128 dimensions of each word in a sequence, then followed by a dense layer with softmax function.

In [96]:
from keras.layers import SpatialDropout1D, LSTM, Dropout

model_lstm = Sequential()

model_lstm.add(Embedding(100000, 128))
model_lstm.add(SpatialDropout1D(0.4))
model_lstm.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model_lstm.add(Dense(1,activation='softmax'))
model_lstm.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model_lstm.summary())
model_lstm.fit(x_train_seq,Y_train, epochs = 10, batch_size=32, verbose = 2)
score,acc = model_lstm.evaluate(x_test_seq, Y_test, verbose = 2, batch_size = 32)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, None, 128)         12800000  
                                                                 
 spatial_dropout1d_2 (Spatia  (None, None, 128)        0         
 lDropout1D)                                                     
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584    
                                                                 
 dense_14 (Dense)            (None, 1)                 129       
                                                                 
Total params: 12,931,713
Trainable params: 12,931,713
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
50/50 - 14s - loss: 0.6481 - accuracy: 0.5025 - 14s/epoch - 276ms/step
Epoch 2/10
50/50 - 11